This code will generate the feature vector, using sub grids to calculate the overlap

In [1]:
import pandas as pd
#pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import cartopy.crs as ccrs
import cartopy
import numpy as np
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')
from joblib import Parallel, delayed

##  Step 0. Throw out fires that have 0 viirs detects on any day. (fires are already filtered to be in the right lat/lon range

In [2]:
# load in the polygons and attributes
fire_polygons = gpd.read_file('fire_polygons_only.geojson')
fire_attributes = pd.read_csv('fire_polygons_attributes.csv')
incidents = pd.read_csv('unique_fires.csv')
print(len(fire_polygons), len(fire_attributes), len(incidents))

#throw out fires that have 0 days with detects
daily_drop_inds = []
incident_drop_inds = []
for item in incidents['Incident Number']:
    pts = fire_attributes['Number of VIIRS points'].values[fire_attributes['Incident Number']==item]
    if len(pts[pts==0])==len(pts):
        #print(item, 'oops all zeros')
        daily_drop_inds = np.append(daily_drop_inds, np.where(fire_attributes['Incident Number']==item)[0])
        incident_drop_inds = np.append(incident_drop_inds, np.where(incidents['Incident Number']==item)[0])

daily_drop_inds = [int(p) for p in daily_drop_inds]
#print(daily_drop_inds)

fire_attributes = fire_attributes.drop(labels= daily_drop_inds)
fire_attributes = fire_attributes.drop(labels='Unnamed: 0', axis=1)
fire_polygons = fire_polygons.drop(labels = daily_drop_inds)
incidents = incidents.drop(labels = incident_drop_inds)
print(len(fire_polygons), len(fire_attributes), len(incidents))

fire_attributes

22280 22280 1354
19137 19137 1036


,Incident Number,Fire Name,Current Day,Lat Fire,Lon Fire,Number of VIIRS points
5,10662684.0,PAINTED WAGON,2019-04-08,33.786944,-112.753333,3.0
6,10662684.0,PAINTED WAGON,2019-04-09,33.786944,-112.753333,0.0
7,10662684.0,PAINTED WAGON,2019-04-10,33.786944,-112.753333,0.0
8,10663171.0,LONE MOUNTAIN,2019-05-26,33.808056,-105.738611,0.0
9,10663171.0,LONE MOUNTAIN,2019-05-27,33.808056,-105.738611,0.0
...,...,...,...,...,...,...
22275,11980587.0,CREEK 5,2020-12-25,33.402646,-117.266795,16.0
22276,11980587.0,CREEK 5,2020-12-26,33.402646,-117.266795,0.0
22277,11980587.0,CREEK 5,2020-12-27,33.402646,-117.266795,0.0
22278,11980587.0,CREEK 5,2020-12-28,33.402646,-117.266795,0.0


## 1. Figure out the start and end date we will use for each fire

In [ ]:
# first check what fraction of the fire lifetimes are 0 days
for item in incidents['Incident Number']:
    pts = fire_attributes['Number of VIIRS points'].values[fire_attributes['Incident Number']==item]
    #print(len(np.where(pts==0)[0]))
    print('The fire lifetime is ', len(np.where(pts!=0)[0])*100/len(pts), '% nonzero detections')

In [3]:
keep_days = []
for item in incidents['Incident Number']:
    pts = fire_attributes['Number of VIIRS points'].values[fire_attributes['Incident Number']==item]
    dys = fire_attributes['Current Day'].values[fire_attributes['Incident Number']==item]
    nonzero = np.where(pts!=0)[0]
    first_day = dys[nonzero[0]]
    last_day = dys[nonzero[len(nonzero)-1]]
    keep_days = np.append(keep_days, np.where((fire_attributes['Incident Number']==item)&(fire_attributes['Current Day']>=first_day)&(fire_attributes['Current Day']<=last_day))[0])

keep_days = [int(p) for p in keep_days]
print(len(keep_days))
fire_attributes = fire_attributes.iloc[keep_days, :]
fire_polygons = fire_polygons.iloc[keep_days, :]
fire_attributes = fire_attributes.reset_index(drop=True) #reset the indices beecause we dropped some
fire_polygons = fire_polygons.reset_index(drop=True) #reset the indices beecause we dropped some

fire_polygons

8161


,geometry
0,"POLYGON ((-112.75455 33.78632, -112.76012 33.7..."
1,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
2,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
3,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
4,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
...,...
8156,"POLYGON ((-112.11669 34.11898, -112.12564 34.1..."
8157,"POLYGON ((-117.03702 33.90632, -117.04484 33.9..."
8158,"POLYGON ((-117.27595 33.37881, -117.27495 33.3..."
8159,"POLYGON ((-117.29549 33.36558, -117.29713 33.4..."


## 2. Loop through the days and get the raw features (temp at different pressure levels, wind, pressure, precip, terrain info) and the labels (QFED PM2.5 from the next day)

In [10]:
# dictionaries that track the variables, dimensions, times and levels
varis = {'/static_maps/static_map_1km.nc4': ['fccs', 'slp', 'asp']}

#0=lat, 1=lon, 2=time, 3=level 
dims = {'/static_maps/static_map_1km.nc4': ['lat', 'lon']}

times = {'TLML':12, 'QLML':12, 'SPEEDLML':12, 'PS':12, 'T':12, 'QV':12, 'biomass':12}
levels = {'T':[500, 700], 'QV':[700]} 
file_suffix = '.nc4'

#pre-allocate the data frame
num_days = len(fire_attributes)
fire_features = gpd.GeoDataFrame({'Incident Number': np.nan*np.zeros(num_days), 'Fire Name': np.nan*np.zeros(num_days), 
                             'Current Day': np.nan*np.zeros(num_days), 'Lat Fire': np.nan*np.zeros(num_days), 
                             'Lon Fire': np.nan*np.zeros(num_days), 'Number of VIIRS points': np.nan*np.zeros(num_days),
                             'fccs': np.nan*np.zeros(num_days),'slp':np.nan*np.zeros(num_days), 
                             'asp':np.nan*np.zeros(num_days),'geometry': np.nan*np.zeros(num_days)}) # pre-allocate the dataframe
#'fccs': np.nan*np.zeros(num_days),'slp': np.nan*np.zeros(num_days),'asp': np.nan*np.zeros(num_days), 
fire_features['Incident Number'] = fire_attributes['Incident Number'].values
fire_features['Fire Name'] = fire_attributes['Fire Name'].values
fire_features['Current Day'] = fire_attributes['Current Day'].values
fire_features['Lat Fire'] = fire_attributes['Lat Fire'].values
fire_features['Lon Fire'] = fire_attributes['Lon Fire'].values
fire_features['Number of VIIRS points'] = fire_attributes['Number of VIIRS points'].values
fire_features['geometry'] = fire_polygons['geometry'].values

#print(fire_features)
unique_days = np.unique(fire_features['Current Day'].values)
print(len(unique_days))
for dd in range(len(unique_days)):

    date=unique_days[dd]
    yr = date[0:4]
    mo = date[5:7]
    dy = date[8:10]
    print(yr, mo,dy)
    #find the polygons for the given day

    for i in varis : #loop over the file names
        #get the path to the file
        path = '/data2/lthapa/'+yr+i
        if '*' in path:
            path = path.replace('*', yr+mo+dy)
        if np.datetime64(date)>=np.datetime64('2020-09-01') and np.datetime64(date)<=np.datetime64('2020-09-30'):
            path = path.replace('400', '401')
            
        #load the file in as a netcdf dataset
        print(path)
        dat = nc.Dataset(path)
        ndims = len(dims[i])
        lat = dat[dims[i][0]][:]
        lon = dat[dims[i][1]][:]
        if len(lat.shape)==1:
            lon, lat = np.meshgrid(lon, lat)
            
        #calculate the areas of the lat/lon grid for the DATASET
        lat_corners, lon_corners = calculate_grid_cell_corners(lat, lon)
        inds = np.where(fire_features['Current Day']==date)[0]
        polys = fire_features['geometry'].iloc[inds].values
        
        for p in range(len(inds)):
            tic = time.time()
            #add a halo to the polygon
            poly = polys[p]
            poly_buff = poly.buffer(1)
            # get the bounds of the buffered polygon
            w,s,e,n = poly_buff.bounds
            #print(w,s,e,n)
            s_ind = np.unique(np.argmin(np.abs(lat-s), axis=0))[0]
            n_ind = np.unique(np.argmin(np.abs(lat-n), axis=0))[0]
            w_ind = np.unique(np.argmin(np.abs(lon-w), axis=1))[0]
            e_ind = np.unique(np.argmin(np.abs(lon-e), axis=1))[0]
            #print(w_ind,s_ind,e_ind,n_ind)
            #lat_sub_inds, lon_sub_inds = np.where((lat>=s)&(lat<=n)&(lon>=w)&(lon<=e))
            #print(lat_sub_inds, lon_sub_inds)
            #remove indices = 0, lat.shape[0], and lat.shape[1]
            lastrow = lat.shape[0]-1
            lastcol = lat.shape[1]-1
        
            #print(lastrow, lastcol)
            

           # print(lat_sub_inds, lon_sub_inds)
    
            
            #tic = time.time()
            overlaps = calculate_overlaps(lat,lon,lat_corners,lon_corners,s_ind,n_ind,w_ind,e_ind, poly) #calculate the overlap between the poly and the lat/lon grid
            #print(np.where(overlaps!=0))
            #toc = time.time()
            #print('Time elapsed for calculating overlaps for polygon:', toc-tic)
            for j in varis[i]: #loop over the variable names, which are lists
                var = dat[j][:]
                #print('the dimensions of', j, 'are', var.shape, 'and', j, 'has', ndims, 'dimensions')
                #select the times and levels we need
                name_in_table = j
                if name_in_table =='fccs': #get the most common fuel type if it's FCCS
                    overlaps[overlaps!=0] =1
                    var_weighted = np.multiply(var,overlaps)
                    val,count = np.unique(var_weighted[var_weighted!=0],return_counts=True)
                    if len(count)==0:
                        var_ave = 0
                    else:
                        var_ave =val[np.argmax(count)]
                    #print(var_ave)          
                else: #do the weighted average
                    var_weighted = np.multiply(var,overlaps)
                    var_ave = np.nansum(var_weighted)
                
                fire_features.loc[inds[p],name_in_table] = var_ave

            toc = time.time()
            print('Time to process polygon=',toc-tic)
        dat.close() 
fire_features

442
2019 02 17
/data2/lthapa/2019/static_maps/static_map_1km.nc4
Time to process polygon= 2.688284397125244
2019 02 18
/data2/lthapa/2019/static_maps/static_map_1km.nc4
Time to process polygon= 2.597132682800293
2019 02 24
/data2/lthapa/2019/static_maps/static_map_1km.nc4
Time to process polygon= 2.6609749794006348
2019 03 06
/data2/lthapa/2019/static_maps/static_map_1km.nc4
Time to process polygon= 1.9590764045715332
2019 03 13
/data2/lthapa/2019/static_maps/static_map_1km.nc4
Time to process polygon= 2.3435051441192627
2019 03 15
/data2/lthapa/2019/static_maps/static_map_1km.nc4
Time to process polygon= 2.666196823120117
2019 03 16
/data2/lthapa/2019/static_maps/static_map_1km.nc4
Time to process polygon= 2.661945343017578
2019 03 17
/data2/lthapa/2019/static_maps/static_map_1km.nc4
Time to process polygon= 2.6274213790893555
2019 03 18
/data2/lthapa/2019/static_maps/static_map_1km.nc4
Time to process polygon= 2.662522792816162
2019 04 05
/data2/lthapa/2019/static_maps/static_map_1km

Time to process polygon= 2.0077943801879883
Time to process polygon= 2.6224396228790283
Time to process polygon= 2.75600266456604
Time to process polygon= 2.562211036682129
Time to process polygon= 2.6587276458740234
2019 06 12
/data2/lthapa/2019/static_maps/static_map_1km.nc4
Time to process polygon= 2.669971466064453
Time to process polygon= 2.685964822769165
Time to process polygon= 2.809520959854126
Time to process polygon= 2.6824023723602295
Time to process polygon= 2.6955513954162598
Time to process polygon= 2.638164520263672
Time to process polygon= 2.7976443767547607
Time to process polygon= 2.6806800365448
Time to process polygon= 2.6726419925689697
2019 06 13
/data2/lthapa/2019/static_maps/static_map_1km.nc4
Time to process polygon= 2.6179161071777344
Time to process polygon= 2.6450839042663574
Time to process polygon= 2.6758475303649902
Time to process polygon= 2.647167444229126
Time to process polygon= 2.7876081466674805
Time to process polygon= 2.565995693206787
Time to pr

Time to process polygon= 2.5706586837768555
Time to process polygon= 2.63838267326355
Time to process polygon= 2.6867923736572266
Time to process polygon= 2.646217107772827
Time to process polygon= 2.7071309089660645
Time to process polygon= 2.6086316108703613
Time to process polygon= 2.7128796577453613
Time to process polygon= 2.764618158340454
Time to process polygon= 2.8135783672332764
Time to process polygon= 2.6077632904052734
Time to process polygon= 2.692835807800293
Time to process polygon= 2.134366989135742
2019 07 03
/data2/lthapa/2019/static_maps/static_map_1km.nc4
Time to process polygon= 2.701500177383423
Time to process polygon= 2.6981964111328125
Time to process polygon= 2.7340714931488037
Time to process polygon= 2.6573238372802734
Time to process polygon= 2.6790459156036377
Time to process polygon= 2.6554741859436035
Time to process polygon= 2.8050074577331543
Time to process polygon= 2.7719764709472656
Time to process polygon= 2.7278716564178467
Time to process polygo

Time to process polygon= 2.6464316844940186
Time to process polygon= 2.665473699569702
Time to process polygon= 2.7246642112731934
Time to process polygon= 2.736212730407715
Time to process polygon= 2.6950440406799316
2019 07 16
/data2/lthapa/2019/static_maps/static_map_1km.nc4
Time to process polygon= 2.6433653831481934
Time to process polygon= 2.770604133605957
Time to process polygon= 2.65097713470459
Time to process polygon= 2.6466686725616455
Time to process polygon= 2.6425604820251465
Time to process polygon= 2.699282169342041
Time to process polygon= 2.6682932376861572
Time to process polygon= 2.7456467151641846
Time to process polygon= 2.6415822505950928
Time to process polygon= 2.6242802143096924
Time to process polygon= 2.676586389541626
Time to process polygon= 2.703071355819702
Time to process polygon= 2.6546905040740967
Time to process polygon= 2.6450181007385254
Time to process polygon= 2.639054298400879
Time to process polygon= 2.6537744998931885
Time to process polygon=

Time to process polygon= 2.8120663166046143
Time to process polygon= 2.9632673263549805
Time to process polygon= 2.950045585632324
Time to process polygon= 2.8070008754730225
Time to process polygon= 2.8976757526397705
Time to process polygon= 2.924347400665283
Time to process polygon= 2.9319427013397217
Time to process polygon= 2.94291615486145
Time to process polygon= 3.0602762699127197
Time to process polygon= 3.05966854095459
Time to process polygon= 2.8064403533935547
Time to process polygon= 2.8131260871887207
Time to process polygon= 2.9206466674804688
Time to process polygon= 2.8972339630126953
Time to process polygon= 2.88142991065979
Time to process polygon= 2.894960641860962
Time to process polygon= 2.9069814682006836
Time to process polygon= 2.920766592025757
2019 07 26
/data2/lthapa/2019/static_maps/static_map_1km.nc4
Time to process polygon= 3.117159366607666
Time to process polygon= 3.0104219913482666
Time to process polygon= 2.954378366470337
Time to process polygon= 2.

...

Time to process polygon= 2.941027879714966
Time to process polygon= 3.4849510192871094
Time to process polygon= 2.909097909927368
Time to process polygon= 3.233071804046631
Time to process polygon= 2.946017026901245
Time to process polygon= 2.960339069366455
Time to process polygon= 3.018808126449585
Time to process polygon= 2.9214584827423096
Time to process polygon= 2.9871292114257812
Time to process polygon= 2.917609691619873
Time to process polygon= 2.9768831729888916
Time to process polygon= 3.054448127746582
Time to process polygon= 2.977741003036499
Time to process polygon= 3.206239700317383
Time to process polygon= 3.149503469467163
2020 10 19
/data2/lthapa/2020/static_maps/static_map_1km.nc4
Time to process polygon= 3.142747402191162
Time to process polygon= 3.0586771965026855
Time to process polygon= 3.1697065830230713
Time to process polygon= 3.0492489337921143
Time to process polygon= 3.080381155014038
Time to process polygon= 3.0256035327911377
Time to process polygon= 3.2

Time to process polygon= 2.622148036956787
Time to process polygon= 2.84360408782959
Time to process polygon= 2.6436266899108887
Time to process polygon= 2.6934521198272705
Time to process polygon= 3.720468521118164
Time to process polygon= 2.836181163787842
2020 10 25
/data2/lthapa/2020/static_maps/static_map_1km.nc4
Time to process polygon= 2.6919472217559814
Time to process polygon= 2.763153553009033
Time to process polygon= 2.6836729049682617
Time to process polygon= 2.7496256828308105
Time to process polygon= 2.7211804389953613
Time to process polygon= 2.7152388095855713
Time to process polygon= 2.795971632003784
Time to process polygon= 2.8111140727996826
Time to process polygon= 2.7206027507781982
Time to process polygon= 2.702422618865967
Time to process polygon= 3.1827142238616943
Time to process polygon= 3.537843704223633
Time to process polygon= 2.680962324142456
Time to process polygon= 2.741478443145752
Time to process polygon= 3.4293723106384277
Time to process polygon= 3

Time to process polygon= 2.7461230754852295
Time to process polygon= 2.8016862869262695
Time to process polygon= 2.7465789318084717
Time to process polygon= 2.726433753967285
Time to process polygon= 3.1963694095611572
Time to process polygon= 3.3826863765716553
Time to process polygon= 2.72310209274292
Time to process polygon= 3.4084887504577637
Time to process polygon= 3.1772329807281494
Time to process polygon= 2.7324368953704834
Time to process polygon= 2.829521656036377
Time to process polygon= 2.690948009490967
Time to process polygon= 2.3878066539764404
Time to process polygon= 2.7107794284820557
Time to process polygon= 2.7495718002319336
2020 11 03
/data2/lthapa/2020/static_maps/static_map_1km.nc4
Time to process polygon= 2.668811321258545
Time to process polygon= 2.6441056728363037
Time to process polygon= 2.6752076148986816
Time to process polygon= 2.702177047729492
Time to process polygon= 2.7157411575317383
Time to process polygon= 2.6025731563568115
Time to process polygo

Time to process polygon= 3.367274761199951
Time to process polygon= 3.4346582889556885
2020 11 21
/data2/lthapa/2020/static_maps/static_map_1km.nc4
Time to process polygon= 2.643749237060547
Time to process polygon= 3.2619497776031494
Time to process polygon= 3.4082484245300293
2020 11 22
/data2/lthapa/2020/static_maps/static_map_1km.nc4
Time to process polygon= 2.681345224380493
Time to process polygon= 3.3655776977539062
Time to process polygon= 3.4202558994293213
2020 11 23
/data2/lthapa/2020/static_maps/static_map_1km.nc4
Time to process polygon= 2.6914010047912598
Time to process polygon= 3.3331172466278076
Time to process polygon= 3.3657708168029785
2020 11 24
/data2/lthapa/2020/static_maps/static_map_1km.nc4
Time to process polygon= 2.713642120361328
Time to process polygon= 3.3702452182769775
Time to process polygon= 3.4589805603027344
2020 11 25
/data2/lthapa/2020/static_maps/static_map_1km.nc4
Time to process polygon= 2.6212680339813232
Time to process polygon= 3.251247167587

,Incident Number,Fire Name,Current Day,Lat Fire,Lon Fire,Number of VIIRS points,fccs,slp,asp,geometry
0,10662684.0,PAINTED WAGON,2019-04-08,33.786944,-112.753333,3.0,307.0,2.0,-3.0,"POLYGON ((-112.75455 33.78632, -112.76012 33.7..."
1,10663171.0,LONE MOUNTAIN,2019-06-09,33.808056,-105.738611,2.0,25.0,10.0,98.0,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
2,10663171.0,LONE MOUNTAIN,2019-06-10,33.808056,-105.738611,0.0,25.0,10.0,98.0,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
3,10663171.0,LONE MOUNTAIN,2019-06-11,33.808056,-105.738611,0.0,25.0,10.0,98.0,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
4,10663171.0,LONE MOUNTAIN,2019-06-12,33.808056,-105.738611,0.0,25.0,10.0,98.0,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
...,...,...,...,...,...,...,...,...,...,...
8156,11979237.0,Trail,2020-11-02,34.147222,-112.117778,2.0,601.0,260.0,2591.0,"POLYGON ((-112.11669 34.11898, -112.12564 34.1..."
8157,11980186.0,SANDERSON,2020-12-13,33.889444,-117.070833,11.0,44.0,42.0,819.0,"POLYGON ((-117.03702 33.90632, -117.04484 33.9..."
8158,11980587.0,CREEK 5,2020-12-24,33.402646,-117.266795,2.0,602.0,27.0,642.0,"POLYGON ((-117.27595 33.37881, -117.27495 33.3..."
8159,11980587.0,CREEK 5,2020-12-25,33.402646,-117.266795,16.0,602.0,75.0,881.0,"POLYGON ((-117.29549 33.36558, -117.29713 33.4..."


In [11]:
fire_features
therest = fire_features[['Incident Number', 'Fire Name', 'Current Day','Lat Fire', 'Lon Fire','Number of VIIRS points', 'fccs','slp', 'asp']]



therest.to_csv('fire_features_fuels.csv')

In [12]:
therest

,Incident Number,Fire Name,Current Day,Lat Fire,Lon Fire,Number of VIIRS points,fccs,slp,asp
0,10662684.0,PAINTED WAGON,2019-04-08,33.786944,-112.753333,3.0,307.0,2.0,-3.0
1,10663171.0,LONE MOUNTAIN,2019-06-09,33.808056,-105.738611,2.0,25.0,10.0,98.0
2,10663171.0,LONE MOUNTAIN,2019-06-10,33.808056,-105.738611,0.0,25.0,10.0,98.0
3,10663171.0,LONE MOUNTAIN,2019-06-11,33.808056,-105.738611,0.0,25.0,10.0,98.0
4,10663171.0,LONE MOUNTAIN,2019-06-12,33.808056,-105.738611,0.0,25.0,10.0,98.0
...,...,...,...,...,...,...,...,...,...
8156,11979237.0,Trail,2020-11-02,34.147222,-112.117778,2.0,601.0,260.0,2591.0
8157,11980186.0,SANDERSON,2020-12-13,33.889444,-117.070833,11.0,44.0,42.0,819.0
8158,11980587.0,CREEK 5,2020-12-24,33.402646,-117.266795,2.0,602.0,27.0,642.0
8159,11980587.0,CREEK 5,2020-12-25,33.402646,-117.266795,16.0,602.0,75.0,881.0


## Helper Functions

In [6]:
#this function calculates the area of each grid in a given grid cell
# LAT and LON should be 2d
def calculate_grid_cell_corners(LAT, LON):
    #we will assume the very edges of the polygons don't touch the boundary of the domain
    lat_corners = (LAT[0:(lat.shape[0]-1),  0:(lat.shape[1])-1] + LAT[1:(lat.shape[0]), 1:(lat.shape[1])])/2
    lon_corners = (LON[0:(lat.shape[0]-1),  0:(lat.shape[1])-1] + LON[1:(lat.shape[0]), 1:(lat.shape[1])])/2
    return lat_corners, lon_corners
    """for ii in range(1,LAT.shape[0]-1):
        for jj in range(1, LAT.shape[1]-1):
            cell_lat = LAT[ii, jj]
            cell_lon = LON[ii, jj]
            SW_lat[ii,jj] = LAT[ii,jj] + (LAT[ii-1,jj-1] - LAT[ii,jj])/2
            SW_lon[ii,jj] = LON[ii,jj] + (LON[ii-1,jj-1] - LON[ii,jj])/2
            SE_lat[ii,jj] = LAT[ii,jj] + (LAT[ii-1,jj-1] - LAT[ii,jj])/2
            SE_lon[ii,jj] = LON[ii,jj] + (LON[ii,jj] - LON[ii-1,jj-1])/2
            NW_lat[ii,jj] = LAT[ii,jj] + (LAT[ii,jj] - LAT[ii-1,jj-1])/2
            NW_lon[ii,jj] = LON[ii,jj] + (LON[ii-1,jj-1] - LON[ii,jj])/2
            NE_lat[ii,jj] = LAT[ii,jj] + (LAT[ii,jj] - LAT[ii-1,jj-1])/2
            NE_lon[ii,jj] = LON[ii,jj] + (LON[ii,jj] - LON[ii-1,jj-1])/2
    return  SW_lat, SW_lon, SE_lat, SE_lon, NW_lat, NW_lon, NE_lat, NE_lon"""

In [7]:
# this function calculates the overlap between the given shape (fire_shape) and given grid (LAT, LON). 
#LAT and LON should be 2D

def calculate_overlaps(lat, lon, lat_corners, lon_corners, lat_start, lat_stop, lon_start, lon_stop, fire_shape):
    overlaps = np.zeros(lat.shape)
    # buffer the fire shape so it has a non zero area
    if fire_shape.area ==0:
        fire_shape = fire_shape.buffer(0.01)
    # find the intersection between the first day's polygon and the  grid
    for ii in range(lat_start, lat_stop):
        for jj in range(lon_start, lon_stop):
            #print('ctr', lat[ii,jj], lon[ii,jj])
            """ SW_lat = lat_corners[ii-1, jj-1]
            SW_lon = lon_corners[ii-1, jj-1]
            SE_lat = lat_corners[ii-1, jj-1]
            SE_lon = lon_corners[ii-1, jj]
            NW_lat = lat_corners[ii, jj-1]
            NW_lon = lon_corners[ii, jj-1]
            NE_lat = lat_corners[ii, jj]
            NE_lon = lon_corners[ii, jj]"""
            poly_cell = Polygon([(lon_corners[ii, jj-1],lat_corners[ii, jj-1]), (lon_corners[ii, jj],lat_corners[ii, jj]), (lon_corners[ii-1, jj],lat_corners[ii-1, jj-1]), (lon_corners[ii-1, jj-1], lat_corners[ii-1, jj-1])])
            overlap_cell = fire_shape.intersection(poly_cell)
            #overlaps are the fraction of the POLYGON in each grid cell
            overlaps[ii,jj] = overlap_cell.area/fire_shape.area
    
    return overlaps

In [ ]:
#geoseries way
        #load in the relevant geometry array
        geojson_path = path.replace('.'+yr+mo+dy+'.nc4', '_grid_polygons.geojson')
        geojson_path = geojson_path.replace('/data2/lthapa/'+yr+'/MERRA2/', './')
        geojson_path = geojson_path.replace('/data2/lthapa/'+yr+'/QFED/', './')
        print(geojson_path)
        grid_polys = gpd.read_file(geojson_path)
        
inds = np.where(grid_polys['geometry'].intersection(poly).area)[0]
            areas = grid_polys['geometry'].intersection(poly).area
            #print(areas[inds])
            #print(areas[inds]/poly.area)
            toc=time.time()
            print('Time elapsed for calculating overlaps for polygon:', toc-tic)
